In [4]:
import numpy as np
from gensim.models import Word2Vec
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Input, Merge, Convolution1D, MaxPooling1D
import ast
import pandas as pd
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from __future__ import print_function
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from functools import reduce
import os
from os.path import basename,isfile
import csv 
from keras.models import model_from_json
from keras import backend as K
sequence_length = 1000 #1980

# Parameters
# ==================================================
#
# Model Variations. See Kim Yoon’s Convolutional Neural Networks for 
# Sentence Classification, Section 3 for detail.

# Model Hyperparameters
embedding_dim = 8
filter_sizes = (3, 4)
num_filters = 128
#dropout_prob = (0.1, 0.75)
dropout_prob = (0.1, 0.35)
hidden_dims = 256 

# Training parameters
batch_size = 32
num_epochs = 40
#val_split = 0.33

done=['sermons', 'houthis']
#done=['ACLU','AEU','alqaeda','AndrewMurray','aqis','asl','ast',
#        'Bahai','boko-haram','DorothyDay','Ghandi','YV','isis',
#        'JohnPiper','LiberalJudaism','MalcolmX','MehrBaba','NaumanKhan',
#        'nusra-front','PastorAnderson','PeterGomes','Rabbinic',
#        'Schizophrenia','SeaShepherds','Shepherd','Stalin','taliban',
#        'Unabomber','Unitarian','WBC','hamza-yusuf','yasir-qadhi',
#        'zaki-naik','suhaib-webb','kkk','white-supremacis']
TEXT_DATA_DIR='arabic-docs'
accuracy=[]
pre_0=[]
rec_0=[]
f1_0=[]
pre_1=[]
rec_1=[]
f1_1=[]
model_variation = 'CNN-rand'  #  CNN-rand | CNN-google | CNN-RPC
#print('Model variation is %s' % model_variation)

# Data Preparatopn
# ==================================================
#
# Load data
#print("Loading data...")
 

#load labels
filePath='arabic-groups-labels.txt'
labels={}
with open(filePath,'r') as intputFile:
        reader=csv.reader(intputFile,delimiter=',')
        for fname,y in reader:
            labels[fname]=int(y)
# load all data
all_sentences = []  # list of text articles
all_labels_index = {}  # dictionary mapping label name to numeric id
all_labels = []  # list of label ids

allFiles=sorted(os.listdir(TEXT_DATA_DIR))
all_docs_labels=[] 
for fname in allFiles:
    fpath = os.path.join(TEXT_DATA_DIR, fname)
    f = open(fpath,'rb')
    all_sentences.append(f.read().decode('utf-8'))
    base_name=basename(fname)
    all_labels_index[base_name] = len(all_labels_index)
    label=list(lbl for file,lbl in labels.items() if base_name.startswith(file))[0]
    all_labels.append(label)
    group=list(file for file,lbl in labels.items() if base_name.startswith(file))[0]
    all_docs_labels.append(group)
    f.close()
    
d = []
for i in all_sentences:
    words2 = text_to_word_sequence(i, lower=True, split=" ")
    d.append(words2)

all_sentences = d

for grp_Name in labels:
    if grp_Name in done:
        continue
   
    train_sentences = []  # list of text articles
    train_labels = []  # list of label ids
    test_sentences = []  # list of text articles
    test_labels = []  # list of label ids
    for j in range(0,len(all_docs_labels)):
        if all_docs_labels[j]==grp_Name:
            test_sentences.append(all_sentences[j])
            test_labels.append(all_labels[j])
        else:
            train_sentences.append(all_sentences[j])
            train_labels.append(all_labels[j])
    if len(test_labels)==0:
        continue
    print (grp_Name)
    vocab = sorted(reduce(lambda x, y: x | y, (set(i) for i in train_sentences)))
    print("vocab len:%i"%len(vocab))
    # Reserve 0 for masking via pad_sequences
    # Reserve 1 for unseen testing tokens
    train_vocab_size = len(vocab) + 2
    train_word_idx = dict((c, i + 2) for i, c in enumerate(vocab))
    
    
    
    train_vocabulary= train_word_idx
    train_vocabulary_inv = vocab 
    train_vocabulary_inv.append("</PAD>")
    
    print ("Training: %i pos, %i neg, Testing: %i post, %i neg"%(train_labels.count(1), train_labels.count(0),
                                                                 test_labels.count(1), test_labels.count(0)))

    itr_accuracy=[]
    itr_pre_0=[]
    itr_rec_0=[]
    itr_f1_0=[]
    itr_pre_1=[]
    itr_rec_1=[]
    itr_f1_1=[]    
    
    oldStart=0
    
    for itr in range(3, 4):# to get better and more robust results, run each fold 10 times
      
        if model_variation=='CNN-google': 
            model_name='GoogleNews-vectors-negative300.bin'
            embedding_model = Word2Vec.load_word2vec_format(model_name, binary=True)
            embedding_weights = [np.array([embedding_model[w] if w in embedding_model\
                                                        else np.random.uniform(-0.25,0.25,embedding_model.vector_size)\
                                                        for w in train_vocabulary_inv])]
        elif model_variation=='CNN-RPC':
            model_name='RPC.300'
            embedding_model = Word2Vec.load(model_name)
            embedding_weights = [np.array([embedding_model[w] if w in embedding_model\
                                                        else np.random.uniform(-0.25,0.25,embedding_model.vector_size)\
                                                        for w in train_vocabulary_inv])]
        elif model_variation=='CNN-rand':
            embedding_weights = None
        else:
            raise ValueError('Unknown model variation')    
             
        
      
    
        
        # Building model
        # ==================================================
        #
        # graph subnet with one input and one output,
        # convolutional layers concateneted in parallel
        graph_in = Input(shape=(sequence_length, embedding_dim))
        convs = []
        for fsz in filter_sizes:
            conv = Convolution1D(nb_filter=num_filters,
                                 filter_length=fsz,
                                 border_mode='valid',
                                 activation='relu',
                                 subsample_length=1)(graph_in)
            pool = MaxPooling1D(pool_length=2)(conv)
            flatten = Flatten()(pool)
            convs.append(flatten)
        
        if len(filter_sizes)>1:
            out = Merge(mode='concat')(convs)
        else:
            out = convs[0]
        
        graph = Model(input=graph_in, output=out)
        
        # main sequential model
        model = Sequential()
        if not model_variation=='CNN-static':
            model.add(Embedding(len(train_vocabulary_inv)+1, embedding_dim, input_length=sequence_length,
                                weights=embedding_weights))
        
        model.add(Dropout(dropout_prob[0], input_shape=(sequence_length, embedding_dim)))
        model.add(graph)
        model.add(Dense(hidden_dims))
        model.add(Dropout(dropout_prob[1]))
        model.add(Activation('relu'))
        model.add(Dense(1))
        model.add(Activation('sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
        #model.add(Activation('sigmoid'))
        #
        
        # Training the model
        X = []
        for i in train_sentences:
            x = [train_word_idx[w] for w in i]
            X.append(x)
        
        X_train = pad_sequences(X,sequence_length)
        
        train_Y = np.asarray(train_labels) 
        model.fit(X_train, train_Y, batch_size=batch_size,nb_epoch=num_epochs, verbose=0 )#
        
        
 
        X = []
        for i in test_sentences:
            x = [train_word_idx[w] if w in train_word_idx else 1 for w in i]
            X.append(x)
        
        X_test = pad_sequences(X,sequence_length)
        test_Y = test_labels
        
        test_count=len(X_test)
        
        probs = model.predict(X_test.reshape(test_count, -1))
         
        pred=(probs>0.5).astype(int).flatten() 
        correct=np.sum(pred==test_Y)
        acc=correct/test_count
        
        CorrectClassifications=0
        PosClassified=0
        PosCorrectClassified=0
        NegClassified=0
        NegCorrectClassified=0 
        #Calculate Accuracy 
        j=0
        for  j in range(0,test_count) :
            true_label=test_Y[j]
            label=pred[j]    
            if(label==true_label):
                CorrectClassifications=CorrectClassifications+1
            if(label==1):
                PosClassified=PosClassified+1
            else:
                NegClassified=NegClassified+1
            if(label==1 and label==true_label):
                PosCorrectClassified=PosCorrectClassified+1 
            if(label==0 and label==true_label):
                NegCorrectClassified=NegCorrectClassified+1 
 
        # precision, recall & F-Measure
        TotalPos=test_Y.count(1)
        TotalNeg=test_Y.count(0)
      
        if PosClassified==0:
            posPre=0
        else:
            posPre=PosCorrectClassified/PosClassified
        if TotalPos==0:
            posRec=0
        else:   
            posRec=PosCorrectClassified/TotalPos
        if (posPre+posRec)==0:
            posF=0
        else: 
            posF= 2*posPre*posRec/(posPre+posRec) 
        if NegClassified==0:
            negPre=0
        else: 
            negPre= NegCorrectClassified/NegClassified 
        if TotalNeg==0:
            negRec=0
        else: 
            negRec =NegCorrectClassified/TotalNeg
        if (negPre+negRec)==0:
            negF=0
        else: 
            negF= 2*negPre*negRec/(negPre+negRec) 
        itr_accuracy.append(acc)
        itr_pre_0.append(negPre)
        itr_rec_0.append(negRec)
        itr_f1_0.append(negF)
        itr_pre_1.append(posPre)
        itr_rec_1.append(posRec)
        itr_f1_1.append(posF)
        K.clear_session()
        print("group %s itr %i, %i/%i, Accuracy:%f"%(grp_Name, itr, CorrectClassifications,test_count,acc))
        print("0 class (%i/%i/%i): precision:%f, recall:%f, fmeasure:%f"%(NegClassified,NegCorrectClassified,TotalNeg,negPre,negRec,negF))
        print("1 class (%i/%i/%i): precision:%f, recall:%f, fmeasure:%f"%(PosClassified,PosCorrectClassified,TotalPos,posPre,posRec,posF))
    
    log_file = open("log_logocv.txt","a") 
    log_file.write("group %s\n"%grp_Name) 
    log_file.write("accuracy\n"+" ".join(['%f' % num for num in itr_accuracy])+"\n") 
    log_file.write("pre 0\n"+" ".join(['%f' % num for num in itr_pre_0])+"\n") 
    log_file.write("rec 0\n"+" ".join(['%f' % num for num in itr_rec_0])+"\n") 
    log_file.write("f1 0\n"+" ".join(['%f' % num for num in itr_f1_0])+"\n") 
    log_file.write("pre 1\n"+" ".join(['%f' % num for num in itr_pre_1])+"\n")  
    log_file.write("rec 1\n"+" ".join(['%f' % num for num in itr_rec_1])+"\n")  
    log_file.write("f1 1\n"+" ".join(['%f' % num for num in itr_f1_1])+"\n") 
    log_file.close()
    
    accuracy.append(np.average(itr_accuracy)) 
    pre_0.append(np.average(itr_pre_0))
    rec_0.append(np.average(itr_rec_0))
    f1_0.append(np.average(itr_f1_0))
    pre_1.append(np.average(itr_pre_1))
    rec_1.append(np.average(itr_rec_1))
    f1_1.append(np.average(itr_f1_1))
    
    print("group %s: , Accuracy:%f"%(grp_Name,np.average(itr_accuracy)))
    print("0 class: precision:%f, recall:%f, fmeasure:%f"%(np.average(itr_pre_0),np.average(itr_rec_0),np.average(itr_f1_0)))
    print("1 class: precision:%f, recall:%f, fmeasure:%f"%(np.average(itr_pre_1),np.average(itr_rec_1),np.average(itr_f1_1)))

print("Average:")  
print("Accuracy:%f"%(np.average(accuracy)))
print("0 class: precision:%f, recall:%f, fmeasure:%f"%(np.average(pre_0),np.average(rec_0),np.average(f1_0)))
print("1 class: precision:%f, recall:%f, fmeasure:%f"%(np.average(pre_1),np.average(rec_1),np.average(f1_1)))

hezbollah
vocab len:52509
Training: 380 pos, 1230 neg, Testing: 678 post, 0 neg
group hezbollah itr 3, 678/678, Accuracy:1.000000
0 class (0/0/0): precision:0.000000, recall:0.000000, fmeasure:0.000000
1 class (678/678/678): precision:1.000000, recall:1.000000, fmeasure:1.000000
group hezbollah: , Accuracy:1.000000
0 class: precision:0.000000, recall:0.000000, fmeasure:0.000000
1 class: precision:1.000000, recall:1.000000, fmeasure:1.000000
Average:
Accuracy:1.000000
0 class: precision:0.000000, recall:0.000000, fmeasure:0.000000
1 class: precision:1.000000, recall:1.000000, fmeasure:1.000000


In [5]:
train_vocabulary


{'وبالموت': 43726,
 'والباغين': 41350,
 'القرآني': 11848,
 'تَقُولُونَ': 21914,
 'ليقول': 34860,
 'وتشرب،': 44382,
 'طيبة': 27895,
 'والتخلي': 41490,
 'أبعدوا': 1815,
 'تجنيد': 19634,
 'المستغفرين،': 13212,
 'يصاحب': 50996,
 'توعز': 21759,
 'ومواقعها': 48215,
 'المقررات،': 13725,
 'فأعطيت': 29308,
 'المسيح': 13317,
 'بغرائزه': 18089,
 'عمّارها': 28781,
 'فالماءُ': 29561,
 'أَنْدَادًا': 3749,
 'خيوطا': 24101,
 '\xa0والتي': 1477,
 'بثباته': 17167,
 'الارض': 6954,
 'جلدتكم': 22511,
 'العظيم': 11215,
 'المركزي': 13102,
 'عفا': 28504,
 'يتحركون': 49957,
 'نَفْسِهِ': 39162,
 'وتفسيق': 44569,
 'منتصرين': 37627,
 'مآلهم\xa0': 35058,
 'ومتسوليين': 47645,
 'مجرى،': 35675,
 'طقماً': 27807,
 'المركز،': 13100,
 'بمعركة': 18524,
 'البخار': 7488,
 'النقيب': 14380,
 'سيدفعون': 26280,
 'تحركات': 19723,
 'يمكر': 51673,
 'يحفظ': 50356,
 'حاجة،': 22835,
 'متفوق': 35495,
 'يؤرق': 49805,
 'الْجِنَّ': 14952,
 'توحيدياً': 21719,
 'اطارات': 5587,
 'مربون': 36144,
 'أجهزتك': 1984,
 'بالتنفيذ': 16319,
 'موطني': 

In [6]:
import numpy as np
import csv
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

 
import os
from os.path import basename,isfile
import csv  
from keras import backend as K
from functools import reduce

# Parameters
# ==================================================
#
# Model Variations. See Kim Yoon’s Convolutional Neural Networks for 
# Sentence Classification, Section 3 for detail.
sequence_length = 1000
# Model Hyperparameters
embedding_dim = 8
filter_sizes = (3, 4)
num_filters = 128
#dropout_prob = (0.1, 0.75)
dropout_prob = (0.1, 0.35)
hidden_dims = 256 

# Training parameters
batch_size = 32
num_epochs = 4
#val_split = 0.33

done=['sermons', 'houthis']
#done=['ACLU','AEU','alqaeda','AndrewMurray','aqis','asl','ast',
#        'Bahai','boko-haram','DorothyDay','Ghandi','YV','isis',
#        'JohnPiper','LiberalJudaism','MalcolmX','MehrBaba','NaumanKhan',
#        'nusra-front','PastorAnderson','PeterGomes','Rabbinic',
#        'Schizophrenia','SeaShepherds','Shepherd','Stalin','taliban',
#        'Unabomber','Unitarian','WBC','hamza-yusuf','yasir-qadhi',
#        'zaki-naik','suhaib-webb','kkk','white-supremacis']
TEXT_DATA_DIR='arabic-docs'
accuracy=[]
pre_0=[]
rec_0=[]
f1_0=[]
pre_1=[]
rec_1=[]
f1_1=[]
model_variation = 'CNN-rand'  #  CNN-rand | CNN-google | CNN-RPC
#print('Model variation is %s' % model_variation)

# Data Preparatopn
# ==================================================
#
# Load data
#print("Loading data...")
 

#load labels
filePath='arabic-groups-labels.txt'
labels={}
with open(filePath,'r') as intputFile:
        reader=csv.reader(intputFile,delimiter=',')
        for fname,y in reader:
            labels[fname]=int(y)
# load all data
all_sentences = []  # list of text articles
all_labels_index = {}  # dictionary mapping label name to numeric id
all_labels = []  # list of label ids

allFiles=sorted(os.listdir(TEXT_DATA_DIR))
all_docs_labels=[] 
for fname in allFiles:
    fpath = os.path.join(TEXT_DATA_DIR, fname)
    f = open(fpath,'rb')
    all_sentences.append(f.read().decode('utf-8'))
    base_name=basename(fname)
    all_labels_index[base_name] = len(all_labels_index)
    label=list(lbl for file,lbl in labels.items() if base_name.startswith(file))[0]
    all_labels.append(label)
    group=list(file for file,lbl in labels.items() if base_name.startswith(file))[0]
    all_docs_labels.append(group)
    f.close()
    
d = []
for i in all_sentences:
    words2 = text_to_word_sequence(i, lower=True, split=" ")
    d.append(words2)

all_sentences = d

for grp_Name in labels:
    if grp_Name in done:
        continue
   
    train_sentences = []  # list of text articles
    train_labels = []  # list of label ids
    test_sentences = []  # list of text articles
    test_labels = []  # list of label ids
    for j in range(0,len(all_docs_labels)):
        if all_docs_labels[j]==grp_Name:
            test_sentences.append(all_sentences[j])
            test_labels.append(all_labels[j])
        else:
            train_sentences.append(all_sentences[j])
            train_labels.append(all_labels[j])
    if len(test_labels)==0:
        continue
    print (grp_Name)
    vocab = sorted(reduce(lambda x, y: x | y, (set(i) for i in train_sentences)))
    print("vocab len:%i"%len(vocab))
    # Reserve 0 for masking via pad_sequences
    # Reserve 1 for unseen testing tokens
    train_vocab_size = len(vocab) + 2
    train_word_idx = dict((c, i + 2) for i, c in enumerate(vocab))
    
    
    
    train_vocabulary= train_word_idx
    train_vocabulary_inv = vocab 
    train_vocabulary_inv.append("</PAD>")
    
    print ("Training: %i pos, %i neg, Testing: %i post, %i neg"%(train_labels.count(1), train_labels.count(0),
                                                                 test_labels.count(1), test_labels.count(0)))

    itr_accuracy=[]
    itr_pre_0=[]
    itr_rec_0=[]
    itr_f1_0=[]
    itr_pre_1=[]
    itr_rec_1=[]
    itr_f1_1=[]    
    
    oldStart=0
    
    for itr in range(3, 4):# to get better and more robust results, run each fold 10 times
      
        if model_variation=='CNN-google': 
            model_name='GoogleNews-vectors-negative300.bin'
            embedding_model = Word2Vec.load_word2vec_format(model_name, binary=True)
            embedding_weights = [np.array([embedding_model[w] if w in embedding_model\
                                                        else np.random.uniform(-0.25,0.25,embedding_model.vector_size)\
                                                        for w in train_vocabulary_inv])]
        elif model_variation=='CNN-RPC':
            model_name='RPC.300'
            embedding_model = Word2Vec.load(model_name)
            embedding_weights = [np.array([embedding_model[w] if w in embedding_model\
                                                        else np.random.uniform(-0.25,0.25,embedding_model.vector_size)\
                                                        for w in train_vocabulary_inv])]
        elif model_variation=='CNN-rand':
            embedding_weights = None
        else:
            raise ValueError('Unknown model variation')    
             
        
      
    
        
        # Building model
        # ==================================================
        #
        # graph subnet with one input and one output,
        # convolutional layers concateneted in parallel
         
        # main sequential model
        model = Sequential()
        if not model_variation=='CNN-static':
            model.add(Embedding(len(train_vocabulary_inv)+1, embedding_dim, input_length=sequence_length,
                                weights=embedding_weights))
 
 
        model.add(Bidirectional(LSTM(64)))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))

        # try using different optimizers and different optimizer configs
        model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

 
        #model.add(Dropout(dropout_prob[0], input_shape=(sequence_length, embedding_dim)))
        #model.add(graph)
        #model.add(Dense(hidden_dims))
        #model.add(Dropout(dropout_prob[1]))
        #model.add(Activation('relu'))
        #model.add(Dense(1))
        #model.add(Activation('sigmoid'))
        #model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
        #model.add(Activation('sigmoid'))
        #
        
        # Training the model
        X = []
        for i in train_sentences:
            x = [train_word_idx[w] for w in i]
            X.append(x)
        
        X_train = pad_sequences(X,sequence_length)
        
        train_Y = np.asarray(train_labels) 
        model.fit(X_train, train_Y, batch_size=batch_size,nb_epoch=num_epochs, verbose=1 )#
        
        
 
        X = []
        for i in test_sentences:
            x = [train_word_idx[w] if w in train_word_idx else 1 for w in i]
            X.append(x)
        
        X_test = pad_sequences(X,sequence_length)
        test_Y = test_labels
        
        test_count=len(X_test)
        
        probs = model.predict(X_test.reshape(test_count, -1))
         
        pred=(probs>0.5).astype(int).flatten() 
        correct=np.sum(pred==test_Y)
        acc=correct/test_count
        
        CorrectClassifications=0
        PosClassified=0
        PosCorrectClassified=0
        NegClassified=0
        NegCorrectClassified=0 
        #Calculate Accuracy 
        j=0
        for  j in range(0,test_count) :
            true_label=test_Y[j]
            label=pred[j]    
            if(label==true_label):
                CorrectClassifications=CorrectClassifications+1
            if(label==1):
                PosClassified=PosClassified+1
            else:
                NegClassified=NegClassified+1
            if(label==1 and label==true_label):
                PosCorrectClassified=PosCorrectClassified+1 
            if(label==0 and label==true_label):
                NegCorrectClassified=NegCorrectClassified+1 
 
        # precision, recall & F-Measure
        TotalPos=test_Y.count(1)
        TotalNeg=test_Y.count(0)
      
        if PosClassified==0:
            posPre=0
        else:
            posPre=PosCorrectClassified/PosClassified
        if TotalPos==0:
            posRec=0
        else:   
            posRec=PosCorrectClassified/TotalPos
        if (posPre+posRec)==0:
            posF=0
        else: 
            posF= 2*posPre*posRec/(posPre+posRec) 
        if NegClassified==0:
            negPre=0
        else: 
            negPre= NegCorrectClassified/NegClassified 
        if TotalNeg==0:
            negRec=0
        else: 
            negRec =NegCorrectClassified/TotalNeg
        if (negPre+negRec)==0:
            negF=0
        else: 
            negF= 2*negPre*negRec/(negPre+negRec) 
        itr_accuracy.append(acc)
        itr_pre_0.append(negPre)
        itr_rec_0.append(negRec)
        itr_f1_0.append(negF)
        itr_pre_1.append(posPre)
        itr_rec_1.append(posRec)
        itr_f1_1.append(posF)
        K.clear_session()
        print("group %s itr %i, %i/%i, Accuracy:%f"%(grp_Name, itr, CorrectClassifications,test_count,acc))
        print("0 class (%i/%i/%i): precision:%f, recall:%f, fmeasure:%f"%(NegClassified,NegCorrectClassified,TotalNeg,negPre,negRec,negF))
        print("1 class (%i/%i/%i): precision:%f, recall:%f, fmeasure:%f"%(PosClassified,PosCorrectClassified,TotalPos,posPre,posRec,posF))
    
    log_file = open("log_logocv.txt","a") 
    log_file.write("group %s\n"%grp_Name) 
    log_file.write("accuracy\n"+" ".join(['%f' % num for num in itr_accuracy])+"\n") 
    log_file.write("pre 0\n"+" ".join(['%f' % num for num in itr_pre_0])+"\n") 
    log_file.write("rec 0\n"+" ".join(['%f' % num for num in itr_rec_0])+"\n") 
    log_file.write("f1 0\n"+" ".join(['%f' % num for num in itr_f1_0])+"\n") 
    log_file.write("pre 1\n"+" ".join(['%f' % num for num in itr_pre_1])+"\n")  
    log_file.write("rec 1\n"+" ".join(['%f' % num for num in itr_rec_1])+"\n")  
    log_file.write("f1 1\n"+" ".join(['%f' % num for num in itr_f1_1])+"\n") 
    log_file.close()
    
    accuracy.append(np.average(itr_accuracy)) 
    pre_0.append(np.average(itr_pre_0))
    rec_0.append(np.average(itr_rec_0))
    f1_0.append(np.average(itr_f1_0))
    pre_1.append(np.average(itr_pre_1))
    rec_1.append(np.average(itr_rec_1))
    f1_1.append(np.average(itr_f1_1))
    
    print("group %s: , Accuracy:%f"%(grp_Name,np.average(itr_accuracy)))
    print("0 class: precision:%f, recall:%f, fmeasure:%f"%(np.average(itr_pre_0),np.average(itr_rec_0),np.average(itr_f1_0)))
    print("1 class: precision:%f, recall:%f, fmeasure:%f"%(np.average(itr_pre_1),np.average(itr_rec_1),np.average(itr_f1_1)))

print("Average:")  
print("Accuracy:%f"%(np.average(accuracy)))
print("0 class: precision:%f, recall:%f, fmeasure:%f"%(np.average(pre_0),np.average(rec_0),np.average(f1_0)))
print("1 class: precision:%f, recall:%f, fmeasure:%f"%(np.average(pre_1),np.average(rec_1),np.average(f1_1)))

hezbollah
vocab len:52509
Training: 380 pos, 1230 neg, Testing: 678 post, 0 neg
Epoch 1/4
1610/1610 [==============================] - 245s - loss: 0.3165 - acc: 0.8857     